 # Adaptación de *English-to-Spanish NMT by fchollet*
 El siguiente modelo se entreno desde cero. Posteriormente se hará el tratamiento con los embeddings' de GloVe.

 * Se implementa el modelo transformador de secuencia a secuencia
 * Entrenamiento para realizar traducciones.
 * Se almacena el modelo preentrenado para el uso futuro.

In [1]:
# We set the backend to TensorFlow. The code works with
# both `tensorflow` and `torch`. It does not work with JAX
# due to the behavior of `jax.numpy.tile` in a jit scope
# (used in `TransformerDecoder.get_causal_attention_mask()`:
# `tile` in JAX does not support a dynamic `reps` argument.
# You can make the code work in JAX by wrapping the
# inside of the `get_causal_attention_mask` method in
# a decorator to prevent jit compilation:
# `with jax.ensure_compile_time_eval():`.
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import layers
from keras import ops
from keras.layers import TextVectorization


2025-03-22 09:12:08.700515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742656328.892058    6337 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742656328.946612    6337 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 09:12:09.358973: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Descargar el conjunto de datos de Anki *English-to-Spanish*

In [2]:
#=text_file = keras.utils.get_file(
    # fname="spa-eng.zip",
    # origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    # extract=True,
# )

In [3]:
#text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"
#text_file = "/content/drive/MyDrive/spa.txt"
text_file = "spa-eng/spa.txt"

Analisis de los datos.

Cada línea del conjunto de datos contiene una oración en ingles y su correspondiente oración en español. La oración en inglés es la *source sequence*(oración fuente) y la oración en español es la *target sequence* (oración destino). Para cada token se antepone "[start]" al inicio y se agrega "[end]" al final de la oración en español.

In [4]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))


Vistazo a los pares de oraciones inglés-español.

In [5]:
for _ in range(5):
    print(random.choice(text_pairs))

("Don't do anything like that again.", '[start] No vuelvas a hacer nada así. [end]')
('Tom leaned forward and tapped the cab driver on the shoulder.', '[start] Tom se inclinó hacia delante y golpeó suavemente en el hombro al conductor del taxi. [end]')
('Who do you want to speak to?', '[start] ¿Con quién quieres hablar? [end]')
('Our ancestors arrived in this country 150 years ago.', '[start] Nuestros ancestros llegaron a este país hace 150 años. [end]')
('Jonas Salk developed the polio vaccine in 1952.', '[start] Jonas Salk desarrolló la vacuna contra la polio en 1952. [end]')


A continuación se dividen los pares de oraciones en datos de prueba y de entrenamiento de forma aleatoria.

In [6]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


# Vectorización

Se utilizaron dos instancias de la capa *TextVectorization* para vectorizar los datos de texto una para inglés y otra para español, para convertir las cadenas originales en secuencias de números enteros donde cada entero representa el índice de una palabra en un vocabulario.

La capa de inglés utilizará la estandarización de cadenas predeterminada (eliminando los caracteres de puntuación) y el esquema de división (dividir por espacios), mientras que la capa de español utilizará una estandarización personalizada para remover carácteres de puntuación propios del lenguaje español.

Nota: En un modelo de traducción automática de producción, no recomendaría eliminar los caracteres de puntuación en ninguno de los dos idiomas. En su lugar, recomendaría convertir cada carácter de puntuación en su propio token, lo cual se podría lograr proporcionando una función de división personalizada a la capa TextVectorization.

In [7]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64
# 6.b
ngrams = 3

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    ngrams=ngrams
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    ngrams=ngrams
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)


I0000 00:00:1742656334.233108    6337 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3108 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1


A continuación, se da formato a los conjuntos de datos.

En cada paso de entrenamiento, el modelo intentará predecir las palabras objetivo N+1 (y posteriores) utilizando la oración fuente y las palabras objetivo de 0 a N.

Para ello, el conjunto de datos de entrenamiento generará una tupla (entradas, objetivos), donde:

* Entradas es un diccionario con las claves encoder_inputs y decoder_inputs. 
* encoder_inputs es la oración fuente vectorizada.
* decoder_inputs es la oración objetivo con las palabras de 0 a N utilizadas para predecir la palabra N+1 (y posteriores) en la oración objetivo.
* Objetivo es la oración objetivo desplazada por un paso: proporciona las siguientes palabras en la oración objetivo, lo que el modelo intentará predecir.

In [8]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


Formas de la secuencia: Se tienen lotes de 64 pares y todas las secuencias tienen una longitud de 20 pasos

In [9]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


2025-03-22 09:12:18.561589: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Construcción del modelo

Este modelo Transformer secuencia a secuencia consta de un *TransformerEncoder* y un *TransformerDecoder* encadenados. 
Para que el modelo reconozca el orden de las palabras, se utiliza una capa de *PositionalEmbedding*.

La secuencia fuente envía al TransformerEncoder el cual genera una nueva representación de la misma. Esta nueva representación pasa al TransformerDecoder, junto con la secuencia objetivo hasta el momento (palabras objetivo de 0 a N). El TransformerDecoder intenta predecir las siguientes palabras en la secuencia objetivo (N+1 y posteriores).

Un detalle clave que lo hace posible es el enmascaramiento causal (véase el método get_causal_attention_mask() en el TransformerDecoder). El TransformerDecoder ve todas las secuencias a la vez, por lo que debemos asegurarnos de que solo utilice la información de los tokens objetivo de 0 a N al predecir el token N+1 (de lo contrario, podría utilizar información del futuro, lo que daría como resultado un modelo que no se pueda utilizar en el momento de hacer la inferencia.

In [10]:
import keras.ops as ops

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        inputs, encoder_outputs = inputs
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


A continuación se ensambla el modelo de extremo a extremo.

In [11]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

# Entrada y salida del encoder y decoder
encoder_inputs = layers.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = layers.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = layers.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# Modelo Transformer final
transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

# Resumen del modelo
#transformer.summary()

In [12]:
# import tensorflow as tf
# print(tf.__version__)

transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None)      │          0 │ decoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decoder │ (None, None, 256) │  5,259,520 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
│                     │                   │            │ not_equal_1[0][0… │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 256) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  3,855,000 │ dropout_3[0][0]   │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

# Entrenando el modelo

Se utiliza la precisión como una forma rápida de monitorear el progreso del entrenamiento con los datos de validación. 
También para la traducción automática suelen utilizar otras métricas como *BLEU*, *ROUGE*.

Para que el modelo converja realmente se debe entrenar durante al menos 30 épocas.

In [16]:
import time
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

# Iniciar el tiempo
start_time = time.time()

# Callbacks para guardar logs, mejores modelos y evitar sobreentrenamiento
csv_logger = CSVLogger("training_logs.csv", append=True)  # Guarda logs en CSV

checkpoint_callback = ModelCheckpoint(
    filepath="transformer_best.keras",  # Guarda el mejor modelo
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)

early_stopping = EarlyStopping(
    monitor="val_loss",  # Detiene si la pérdida no mejora
    patience=5,  # Número de épocas sin mejora antes de detener
    restore_best_weights=True  # Restaura los mejores pesos encontrados
)

# 6.a Usar más de 30 épocas
new_epochs = 1

# 6.c Cambiar la tasa de aprendizaje
lr = 1e-4  # Prueba con 1e-3, 5e-4, 1e-5, etc.
# 6.d Cambiar el optimizador
optimizer = Adam(learning_rate=lr) # probar otros
# optimizer = RMSprop(learning_rate=1e-4)

transformer.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["Accuracy"])


# Entrenamiento
history = transformer.fit(
    train_ds,
    epochs=new_epochs,
    validation_data=val_ds,
    callbacks=[csv_logger, checkpoint_callback, early_stopping]
)

# Guardar modelo completo
transformer.save("transformer_updated.keras")

# Guardar solo los pesos
#transformer.save_weights("transformer_weights.h5")

# Medir el tiempo total
elapsed_time = time.time() - start_time
print(f"Training completed in {elapsed_time:.2f} seconds")


W0000 00:00:1742657035.421724    6399 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2025-03-22 09:23:59.835259: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'copy_fusion_1', 464 bytes spill stores, 484 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_3', 104 bytes spill stores, 104 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_1', 104 bytes spill stores, 104 bytes spill loads



 946/1302 ━━━━━━━━━━━━━━━━━━━━ 1:01 173ms/step - Accuracy: 0.5059 - loss: 2.8773

W0000 00:00:1742657203.690929    6398 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


 947/1302 ━━━━━━━━━━━━━━━━━━━━ 1:03 178ms/step - Accuracy: 0.5059 - loss: 2.8772

2025-03-22 09:26:48.738803: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'copy_fusion_1', 464 bytes spill stores, 484 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



...

W0000 00:00:1742657271.641143    6398 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1742657279.134741    6398 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


1302/1302 ━━━━━━━━━━━━━━━━━━━━ 254s 186ms/step - Accuracy: 0.5094 - loss: 2.8422 - val_Accuracy: 0.5561 - val_loss: 2.2841


/home/luna/.local/share/virtualenvs/Martinez_Selene-BhocuyRJ/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


Training completed in 260.13 seconds


KeyError: 'val_accuracy'

In [19]:
# Mostrar métricas finales
# print(history.history.keys())
final_acc = history.history["Accuracy"][-1]
final_val_acc = history.history["val_Accuracy"][-1]
final_loss = history.history["loss"][-1]
final_val_loss = history.history["val_loss"][-1]

print(f"Final Training Accuracy: {final_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Final Training Loss: {final_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

Final Training Accuracy: 0.5220
Final Validation Accuracy: 0.5561
Final Training Loss: 2.7112
Final Validation Loss: 2.2841


# Decodificación de oraciones de prueba
A continuación se demostrará cómo traducir oraciones nuevas en inglés. Simplemente introducimos en el modelo la oración vectorizada en inglés y el token de destino "[start]". Luego, generamos repetidamente el siguiente token hasta llegar al token "[end]".

In [20]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer(
            {
                "encoder_inputs": tokenized_input_sentence,
                "decoder_inputs": tokenized_target_sentence,
            }
        )

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, translated)


She has seven sons. [start] ella tiene las personas [end]
Don't expect others to help you. [start] no te [UNK] a los ayuda [end]
He gave me not just advice, but money as well. [start] Él me dio no le dio dinero pero dinero pero a la dinero [end]
There must've been a tacit understanding between them. [start] tiene que haber [UNK] un [UNK] entre ellos [end]
This data is for my thesis. [start] este [UNK] es por mi [UNK] [end]
How was your summer? [start] cómo fue tu verano [end]
I didn't know that. [start] no sabía eso [end]
He rolled over in his sleep. [start] Él se [UNK] en su trabajo [end]
We hit a deer. [start] [UNK] a un [UNK] [end]
You can start right now. [start] puedes [UNK] ahora [end]


# Metricas

* 6.e Cambiar las métricas. 
* 6.f Se BLEU.
* 6.g Se utiliza Rouge

In [22]:
!pip install rouge-score

In [26]:
from rouge_score import rouge_scorer

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

for test_pair in test_pairs[:10]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]
    translated_sentence = decode_sequence(input_sentence)
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    scores = rouge_scorer.score(reference_sentence, translated_sentence)
    print(f"Input: {input_sentence}")
    print(f"Reference: {reference_sentence}")
    print(f"Translated: {translated_sentence}")
    print(f"ROUGE-1 Precision: {scores['rouge1'].precision:.4f}, Recall: {scores['rouge1'].recall:.4f}, F1-score: {scores['rouge1'].fmeasure:.4f}")
    print(f"ROUGE-2 Precision: {scores['rouge2'].precision:.4f}, Recall: {scores['rouge2'].recall:.4f}, F1-score: {scores['rouge2'].fmeasure:.4f}")
    print("-" * 50)

Input: Tom raises Arabian horses.
Reference: [start] Tom cría caballos árabes. [end]
Translated: [start] tom [UNK] los [UNK] los estudiantes [end]
ROUGE-1 Precision: 0.3750, Recall: 0.4286, F1-score: 0.4000
ROUGE-2 Precision: 0.1429, Recall: 0.1667, F1-score: 0.1538
--------------------------------------------------
Input: It's easy for monkeys to climb trees.
Reference: [start] Para los monos es sencillo trepar árboles. [end]
Translated: [start] es fácil para [UNK] para [UNK] [end]
ROUGE-1 Precision: 0.4444, Recall: 0.4444, F1-score: 0.4444
ROUGE-2 Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
--------------------------------------------------
Input: After the accident, the police told the crowd to keep back.
Reference: [start] Tras el accidente, la policía le dijo a la multitud que se alejaran. [end]
Translated: [start] después de el accidente de la policía se le dijo a la policía [end]
ROUGE-1 Precision: 0.6667, Recall: 0.7500, F1-score: 0.7059
ROUGE-2 Precision: 0.3529, Recal

In [33]:
import pandas as pd

# Assuming 'scores' is a list of dictionaries, each containing the ROUGE scores for a test pair.
# If not, adjust accordingly based on your actual data structure.

# Crear una lista de diccionarios para almacenar los resultados
rows = []
for test_pair in test_pairs[:10]:
  input_sentence = test_pair[0]
  reference_sentence = test_pair[1]
  translated_sentence = decode_sequence(input_sentence)
  translated_sentence = (
      translated_sentence.replace("[PAD]", "")
      .replace("[START]", "")
      .replace("[END]", "")
      .strip()
  )
  scores = rouge_scorer.score(reference_sentence, translated_sentence)

  row = {
      "Input": input_sentence,
      "Reference": reference_sentence,
      "Translated": translated_sentence,
      "ROUGE-1 Precision": scores['rouge1'].precision,
      "ROUGE-1 Recall": scores['rouge1'].recall,
      "ROUGE-1 F1-score": scores['rouge1'].fmeasure,
      "ROUGE-2 Precision": scores['rouge2'].precision,
      "ROUGE-2 Recall": scores['rouge2'].recall,
      "ROUGE-2 F1-score": scores['rouge2'].fmeasure,
  }
  rows.append(row)

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(rows)

# Export the DataFrame to a CSV file
df.to_csv("rouge_scores_II.csv", index=False)
# df.to_csv("/content/drive/MyDrive/ColabNotebooks/rouge_scores.csv", index=False)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Crear una lista de diccionarios para almacenar los resultados
rows = []
smoother = SmoothingFunction().method1  # Suavizado para evitar BLEU=0 en frases cortas

for test_pair in test_pairs[:10]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]
    translated_sentence = decode_sequence(input_sentence)
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    # Tokenizar las oraciones
    reference_tokens = [reference_sentence.split()]
    translated_tokens = translated_sentence.split()

    # Calcular BLEU con 1-gram, 2-gram, 3-gram y 4-gram
    bleu1 = sentence_bleu(reference_tokens, translated_tokens, weights=(1, 0, 0, 0), smoothing_function=smoother)
    bleu2 = sentence_bleu(reference_tokens, translated_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoother)
    bleu3 = sentence_bleu(reference_tokens, translated_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoother)
    bleu4 = sentence_bleu(reference_tokens, translated_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoother)

    row = {
        "Input": input_sentence,
        "Reference": reference_sentence,
        "Translated": translated_sentence,
        "BLEU-1": bleu1,
        "BLEU-2": bleu2,
        "BLEU-3": bleu3,
        "BLEU-4": bleu4,
    }
    rows.append(row)

# Crear DataFrame y exportar a CSV
df = pd.DataFrame(rows)
# df.to_csv("/content/drive/MyDrive/ColabNotebooks/bleu_scores.csv", index=False)
df.to_csv("bleu_scores.csv", index=False)
